## Исследование надёжности заёмщиков

План:
1. Изучим данные разными методами, чтобы проверить пропуски, типы данных и познакомиться с самими данными, которые доступны для исследования.
1. Предобработаем данные (разберёмся с пропусками и возможными дублями), приведём их к нужному регистру и типу данных.
1. Категоризируем заёмщиков по:
    - количеству детей,
    - семейному положению,
    - уровню дохода,
    - целям кредита.
1. Проанализируем данные. Ответим на вопросы:
    - Есть ли зависимость между наличием детей и возвратом кредита в срок.
    - Есть ли зависимость между семейным положением и возвратом кредита в срок
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок.
    - Как разные цели кредита влияют на его возврат в срок.
1. Сформируем выводы о поведении заёмщиков банка.    

### Шаг 1. Изучим общую информацию

In [1]:
import pandas as pd

In [2]:
# загружаем файл с данными

data = pd.read_csv('/datasets/data.csv')

In [3]:
# общая информация по таблице

data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# посмотрим на количественные значения с помощью describe

data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Посмотрим, какие уникальные значения есть в качественных признаках и какое количество по каждому из них - пригодится в будушем на шаге чистки дубликатов.

In [5]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [6]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [7]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [9]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

***Вывод***

1. Есть пропуски в столбцах:
  - days_employed - стаж,
  - total_income - информация о доходе,


2. Несоответствие типов данных:
  - days_employed и total_income - float64, вряд ли стаж и доход могут быть дробным числом, необходимо перевести в тип int.


3. Стоит обратить внимание на несколько моментов: 
  - отрицательные значения в трудовом стаже (days_employed), 
  - разные регистры в образовании (education) - запомним на будущее,
  - у одного клиента вместо пола стоит значение XNA (gender) - изменить его на что-то конкретное мы не можем, поэтому т.к. он такой один и на результаты не повлияет, не будем его трогать,
  - очевидных скрытых дубликатов нет, кроме формулировок целей.  

## Шаг 2. Предобработка данных

### Обработка пропусков

In [10]:
# найдём все пропуски

data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [11]:
# удаляем пропущенные значения в стаже

data.dropna(subset = ['days_employed'], inplace = True) 

In [12]:
# найдём все уникальные типы занятости и запишем их в перменную

income_type_unique = data['income_type'].unique() 

# заполним пропуски в совокупном доходе средними значениями по каждому типу заности, выделенному в предыдущем действии

for unitype in income_type_unique:
    total_income_mean = data[data['income_type'] == unitype]['total_income'].mean() 
    data.loc[(data['income_type'] == unitype) & (data['total_income'].isnull()), 'total_income'] = total_income_mean

In [13]:
# проверяем, что пропусков больше нет

data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

***Вывод***

В 2174 строках из 21525 строк пропущены значения стажа и совокупного дохода.
Вероятнее всего, эти две строки не заполнены по тем клиентам, по которым данные не подгрузились в систему. 

Т.к. нас интересуют другие заполненные данные по этим клиентам для ответа на вопросы про возврат кредита в зависимости от кол-ва детей и семейного положения, то не буду удалять полностью все строки с отсутствующими значениями по стажу и доходу. 

Если бы это были неработающие сейчас люди с нулевым доходом (пенсионер, безработный, студент, в декрете), у некоторых из них был бы стаж.
Если бы это были люди с нулевым стажем (студент, предположительно "в декрете" и "безработный"), то их не может быть так много (в сумме они дают всего 4 человека). 

Заполнять отсутствующие значения средним стажем - некорректно, т.к. в данных есть отрицательные числа и слишком большие - явно не соответствует действительности. В будущем эта метрика нам пока не пригодится, подсчёт данных из этого столбца ни на что влиять не будет, поэтому просто удалила пропущенные значения стажа, не обрабатывая этот столбец.

А вот совокупный доход нам понадобится для ответа на один из вопросов. Его пропуски заполнила средними значениями по каждой категории.

### Замена типа данных

In [14]:
# посмотрим типы данных

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


In [15]:
# меняем тип данных для столбцов days_employed и total_income: из вещественного переводим в целочисленный

data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')

In [16]:
# проверем тип данных и вывод первых 5 строк

display(data.info())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


***Вывод***

Теперь информацию в отредактированных столбцах легче воспринимать. Совокупный доход понадобится нам в следующих шагах.

### Обработка дубликатов

In [17]:
print(data.duplicated().sum()) # считаем дубли - 54

# как мы выяснили в первом шаге, в столбце education используются разные регистры - приведём все значения в нём к нижнему 
data['education'] = data['education'].str.lower()

print(data.duplicated().sum()) # повторно считаем дубли - 71

data = data.drop_duplicates().reset_index(drop=True) # удаляем дубликаты
print(data.duplicated().sum()) # перепроверяем дубли - 0

0
0
0


***Вывод***

Привели значения столбца education к нижнему регистру.
Таким образом получилось найти ещё 17 полных дубликатов. Дропнули их методом с перезаписью .drop_duplicates().reset_index(drop=True).

Возможные причины появления дубликатов: технические ошибки и повторные заполнения (возможно, одни и те же люди несколько раз подавали заявки на кредит).

Остальные столбцы на уникальность мы уже проверяли в первом шаге - скрытых дублей в них не было (столбец с целями рассмотрим далее отдельно, когда будем его категоризировать).

### Лемматизация

In [18]:
# импортируем библиотеку для лемматизации

from pymystem3 import Mystem
m = Mystem()

In [19]:
# переводим колонку целей в список

purpose = data['purpose'].tolist() 

# лемматизируем этот список с помощью цикла

lemmas = []
for row in purpose:
    lemmas += m.lemmatize(row)

# посчитаем кол-во лемм

from collections import Counter
print(Counter(lemmas))

Counter({' ': 30248, '\n': 19351, 'недвижимость': 5723, 'покупка': 5353, 'жилье': 4035, 'автомобиль': 3897, 'образование': 3597, 'с': 2619, 'операция': 2334, 'свадьба': 2099, 'на': 2016, 'свой': 2013, 'строительство': 1701, 'высокий': 1243, 'получение': 1179, 'коммерческий': 1178, 'для': 1158, 'жилой': 1126, 'сделка': 844, 'дополнительный': 810, 'заниматься': 797, 'сыграть': 693, 'проведение': 685, 'сдача': 588, 'семья': 570, 'собственный': 560, 'со': 559, 'ремонт': 542, 'подержать': 437, 'подержанный': 435, 'приобретение': 419, 'профильный': 389})


***Вывод***

Судя по полученным результатам, больше всего кредитов берут на недвижимость.

Примерно одинаково - на авто и образование. 

4-й в топе стоит свадьба.

### Категоризация данных

**Количество детей**

In [20]:
# напишем функцию для определения категорий клиентов по количеству детей

def category_children(children):
    if children == 0:
        return 'нет детей'
    elif children == 1:
        return '1 ребёнок'
    elif children <= 3:
        return '2-3 ребёнка'
    elif children > 3:
        return 'больше 3-х детей'


# применим функцию к датасету и создадим новый столбец с новыми категориями

data['category_children'] = data['children'].apply(category_children)

# посчитаем, какое количество входит в каждую категорию

data['category_children'].value_counts()

нет детей           12710
1 ребёнок            4343
2-3 ребёнка          2189
больше 3-х детей      109
Name: category_children, dtype: int64

Мы выделили четыре категории по количеству детей: без детей, 1 ребёнок (самый частый вариант для России в семьях с детьми), 2-3 ребёнка (второй по популярности), многодетные (больше 3-х детей).

***Вывод***

Видим, что больше чем у половины наших клиентов нет детей.

**Семейное положение**

Сначала составии словарь, в котором каждому семейному статусу соответствует свой id. Это действие поможет написать функцию для категоризации заёмщиков.

In [21]:
# создадим словарь со статусом семейного положения и его айди

family_status_dict = data[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) # удалили дубли

# смотрим на словарь, отсортированный по типу айди

family_status_dict.sort_values(by = 'family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [22]:
# создадим функцию для определения категорий клиентов по семейному положению

def category_family_status(family_status_id):
    if family_status_id == 0:
        return 'женат / замужем'
    if family_status_id == 1:
        return 'гражданский брак'
    if family_status_id == 2:
        return 'не женат / не замужем'
    if family_status_id == 3:
        return 'в разводе'
    if family_status_id == 4:
        return 'вдовец / вдова'

# применим функцию к датасету и создадим новый столбец с новыми категориями

data['category_family_status'] = data['family_status_id'].apply(category_family_status)

# посчитаем, какое количество входит в каждую категорию

data['category_family_status'].value_counts()

женат / замужем          11143
гражданский брак          3735
вдовец / вдова            2525
в разводе                 1083
не женат / не замужем      865
Name: category_family_status, dtype: int64

***Вывод***

Мы поделили заёмщиков по категориям.

Почти половину кредитов берут те, кто состоит в браке.

**Уровень дохода**

In [23]:
# посмотрим на столбец совокупного дохода методом describe(), чтобы понять как распределились данные (средние значения и т.д.)

data['total_income'].describe().astype('int')

count      19351
mean      167421
std       102971
min        20667
25%       103053
50%       145017
75%       203434
max      2265604
Name: total_income, dtype: int64

Учитывая, что в данных есть "выбросы" с очень большими значениями по отношению к остальным (максимальный доход 2265604), среднее значение может быть отклонено от реальных результов, поэтому ориентируемся на перцентили и медиану (50%):
- 25% - 103053 - возьмём в качестве границы для низкого дохода,
- 50% - 145017 - граница для среднего дохода,
- 75% - 203434 - граница для высокого дохода.

Выделим 4 категории: низкий, средний, высокий и очень высокий.

In [24]:
# создадим функцию для группировки клиентов по уровню дохода

def category_income(total_income):
    if total_income <= 103053:
        return 'низкий доход'
    elif total_income <= 145017:
        return 'средний доход'
    elif total_income <= 203434:
        return 'высокий доход'
    else:
        return 'очень высокий доход'
    
# формируем новый столбец с названиями типов дохода 

data['category_income'] = data['total_income'].apply(category_income)

# проверим на первых строках, что всё получилось и мы видим распределение дохода по категориям

data['category_income'].head()

0    очень высокий доход
1          средний доход
2          высокий доход
3    очень высокий доход
4          высокий доход
Name: category_income, dtype: object

**Цели кредита**

По ключевым словам из столбца "цель кредита" распределии всех клиентов в основные крупные категории. Затем посчитаем количество в каждой.

In [25]:
# создадим функцию, которая категоризирует цели кредита по основе слова из столбца целей

def category_purpose(purpose):
    if 'недвиж' in purpose:
        return 'недвижимость' 
    if 'жиль' in purpose:
        return 'недвижимость'
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'автом' in purpose:
        return 'автомобиль'
    if 'образ' in purpose:
        return 'образование'

# создадим новый столбец с категориями целей, выделенными с помощью функции

data['category_purpose'] = data['purpose'].apply(category_purpose)

# проверяем

data['category_purpose'].value_counts()

недвижимость    9758
автомобиль      3897
образование     3597
свадьба         2099
Name: category_purpose, dtype: int64

***Вывод***

Видим, что кредит на недвижимость берут в 3 раза чаще, чем на автомобиль и образование.

### Общие выводы по разделу

1. Больше половины кредитов взяли клиенты, у которых нет детей.
2. Почти половину кредитов берут те, кто состоит в браке.
3. Чаще всего клиенты банка берут кредиты на недвижимость и жильё.

## Шаг 3. Анализируем полученную информацию

### Есть ли зависимость между наличием детей и возвратом кредита в срок

In [26]:
# используем созданный ранее столбец с категориями клиентов по количеству детей 'category_children'
# сгруппируем датасет по нему, а также с помощью метода agg посчитаем количество кредиторов в каждой категории и сумму долгов

data_children = data.groupby('category_children').agg({'children':'count', 'debt':'sum'}).reset_index()

# создадим новый столбец, который будет показывать процент невозврата кредита в каждой категории

data_children['debt_ratio_%'] = data_children['debt'] / data_children['children'] * 100
data_children.round(2).sort_values('debt_ratio_%')

,category_children,children,debt,debt_ratio_%
3,нет детей,12710,952,7.49
1,2-3 ребёнка,2189,200,9.14
0,1 ребёнок,4343,408,9.39
2,больше 3-х детей,109,11,10.09


***Вывод по количеству детей***

Самый высокий процент невозврата у семей, которые растят больше **трёх детей**. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок

In [27]:
# используем созданный ранее столбец с категориями клиентов, распределённых по семейному статусу 'category_family_status'
# сгруппируем датасет по нему, а также с помощью метода agg посчитаем количество кредиторов в каждой категории и сумму долгов

data_family = data.groupby('category_family_status').agg({'family_status_id':'count', 'debt':'sum'}).reset_index()

# создаю новый столбец, который будет показывать процент невозврата кредита в каждой категории

data_family['debt_ratio_%'] = data_family['debt'] / data_family['family_status_id'] * 100
data_family.round(2).sort_values('debt_ratio_%')

,category_family_status,family_status_id,debt,debt_ratio_%
4,не женат / не замужем,865,56,6.47
0,в разводе,1083,76,7.02
3,женат / замужем,11143,846,7.59
2,гражданский брак,3735,339,9.08
1,вдовец / вдова,2525,254,10.06


***Вывод по семейному положению***

По процентному соотношению невозвратов видим, что выше всех оно у **вдовцов**, на втором месте идут пары, состоящие в гражданском браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [28]:
# используем созданный ранее столбец с категориями клиентов, распределённых по совокупному доходу 'category_income'
# сгруппируем датасет по нему, а также с помощью метода agg посчитаем количество кредиторов в каждой категории и сумму долгов

data_income = data.groupby('category_income').agg({'total_income':'count', 'debt':'sum'}).reset_index()

# создаю новый столбец, который будет показывать процент невозврата кредита в каждой категории

data_income['debt_ratio_%'] = data_income['debt'] / data_income['total_income'] * 100
data_income.round(2).sort_values('debt_ratio_%')

,category_income,total_income,debt,debt_ratio_%
2,очень высокий доход,4838,341,7.05
1,низкий доход,4838,383,7.92
3,средний доход,4838,421,8.70
0,высокий доход,4837,426,8.81


***Вывод по уровню дохода***

Клиенты с **высоким доходом** показывают самый высокий процент невозврата.

На втором месте люди со средним доходом, разница между ними всего 0,1.

### Как разные цели кредита влияют на его возврат в срок

In [29]:
# используем созданный ранее столбец с категориями клиентов, распределённых по целям кредита 'category_purpose'
# сгруппируем датасет по нему, а также с помощью метода agg посчитаем количество кредиторов в каждой категории и сумму долгов

data_purpose = data.groupby('category_purpose').agg({'purpose':'count', 'debt':'sum'}).reset_index()

# создаю новый столбец, который будет показывать процент невозврата кредита в каждой категории

data_purpose['debt_ratio_%'] = data_purpose['debt'] / data_purpose['purpose'] * 100
data_purpose.round(2).sort_values('debt_ratio_%')

,category_purpose,purpose,debt,debt_ratio_%
1,недвижимость,9758,715,7.33
3,свадьба,2099,158,7.53
2,образование,3597,331,9.20
0,автомобиль,3897,367,9.42


***Вывод по целям кредита***

Почти половина от всех взятых кредитов - это кредиты на недвижимость. 

Если посмотрим на процент кредитов, которые не вернули, то на **автомобили** приходится 9,42% - это выше, чем у всех остальных.
С небольшим отрывом в 0,2 на втором месте по невозврату идут кредиты на образование.

## Шаг 4. Общий вывод

1. Самый высокий процент невозврата у семей, имеющих больше **трёх детей** (10%). На втором месте - семьи ***с одним ребёнком*** (9,4%). Наличие детей создаёт ощутимую нагрузку на семейный бюджет. Чем больше детей, тем труднее выплачивать кредит. 

Это подтверждается также и тем фактом, что клиенты без детей возвращают кредит лучше всех (процент невозврата 7,5%).


2. Доля невозвратов в зависимости от семейного статуса выше всех у **вдовцов** (10%). Можно было бы предположить, что труднее всего возвращать кредит одиноким, но в противовес им лучше всех справляются с выплатами те, кто не состоит в браке (незамужние - 6,5% и разведённые - 7%). 

Возможно, что кредит брали на достаточно длительный период из расчёта, что выплачивать его будут из двух источников доходов. Также, возможно, что у вдовцов на попечении остаются дети, а как мы выяснили в первом пункте, наличие детей снижает способность к выплате долга.

На втором месте по невозвратам идут пары, состоящие в ***гражданском браке*** (9%). Видимо, отсутствие официального брака влияет на клиентов "расслабляюще", у таких пар нет обязательств друг перед другом, нет стимула "помогать своей семье". Кредит они выплачивают неохотно.

3. Клиенты с **высоким доходом** показывают самый высокий процент невозврата (8,8%). На втором месте люди со ***средним доходом***, разница между ними всего 0,1 (8,7%).

Предполагаю, что именно благодаря своим доходам клиенты берут кредиты в надежде без труда их выплатить. Они не привыкли в чём-то нуждаться, поэтому пользуются кредитами для удовлетворения своих потребностей. Возможно, что в итоге оказывается так, что кредитов набрано слишком много, либо они взяты под те цели, которые оказались с течением времени не нужны.

Лучше всех возвращают кредиты клиенты с очень высоким доходом (7%). Это может объясняться тем, что таким людям в целом не нужны кредиты и если они их всё-таки берут, то могут почти без проблем вернуть благодаря своим доходам. 

4. Интересный факт: почти половина от всех взятых кредитов - это кредиты на недвижимость. Это понятно, потому что из перечисленных категорий на недвижимость требуются гораздо более внушительные суммы, и люди обращаются в банк за кредитом. Также в нашей стране принято покупать жильё, а не брать в аренду, как, например, в Европе или Штатах.
Возвращают такие кредиты лучше всех (процент невозврата самый низкий - 7,3%). В отличие от других целей наличие жилья является гораздо более критичным для человека, чем автомобиль или траты на свадьбу. 

Если посмотрим на процент кредитов, которые не вернули, то на **автомобили** приходится 9,4% - это выше, чем у всех остальных. От автомобиля клиенты отказываются легче всего, для большинства людей он не является чем-то крайне необходимым и они вполне могут обойтись без него, если понимают, что выплачивать кредит им тяжело.

С небольшим отрывом в 0,2 на втором месте по невозврату идут кредиты на ***образование*** (9,2%). Это интересный момент, т.к. в России по-прежнему ценят наличие высшего образования. Возможно, что часть из этих кредитов взяли на дополнительное образование (второе высшее, аспирантура, курсы повышения квалификации), от которого в итоге отказались. Также, возможно, что кредиты брали те клиенты, у которых не было иного способа платить за своё образование ввиду низких доходов, и в итоге по этой же причине они не смогли выплатить свой кредит.

